In [4]:
# Author: Xianhui
#

from __future__ import print_function
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## multi-class classification with Keras and TensorFlow
https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [5]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

C:\Users\cyret\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [6]:
_ = load_iris()
X, y = _.data, _.target
print(X.shape, y.shape)
num_classes = len(np.unique(y))
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y[:5]

(150, 4) (150,)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

In [8]:
def base_model(input_dim=4, num_classes=3):
    '''
    set up keras neural network for multi-class
    input_dim: number of features
    num_classes: output layer units, for multi-class > 2
    '''
    model = Sequential()
    model.add(Dense(units=16, input_dim=input_dim))
    model.add(Activation('relu'))
    # model.add(Dense(units=16))
    # model.add(Activation('softmax'))
    # model.add(Dense(units=16))
    # model.add(Activation('sigmoid'))
    model.add(Dense(num_classes, activation='softmax'))  # 3 units for 3 classes
    adam = Adam(lr=0.001)
    model.compile(optimizer=adam, loss=binary_crossentropy, metrics=['accuracy'])
    return model

In [9]:
model = base_model()
model.fit(X_train, y_train, epochs=150, batch_size=32, validation_split=0.1, verbose=1)

Train on 90 samples, validate on 10 samples
Epoch 1/150
90/90 [==============================] - 0s 3ms/step - loss: 1.3083 - acc: 0.5481 - val_loss: 1.4372 - val_acc: 0.4667
Epoch 2/150
90/90 [==============================] - 0s 77us/step - loss: 1.2483 - acc: 0.5481 - val_loss: 1.3744 - val_acc: 0.4667
Epoch 3/150
90/90 [==============================] - 0s 77us/step - loss: 1.1949 - acc: 0.5481 - val_loss: 1.3168 - val_acc: 0.4667
Epoch 4/150
90/90 [==============================] - 0s 110us/step - loss: 1.1429 - acc: 0.5481 - val_loss: 1.2669 - val_acc: 0.4667
Epoch 5/150
90/90 [==============================] - 0s 94us/step - loss: 1.0988 - acc: 0.5481 - val_loss: 1.2243 - val_acc: 0.4667
Epoch 6/150
90/90 [==============================] - 0s 105us/step - loss: 1.0641 - acc: 0.5630 - val_loss: 1.1872 - val_acc: 0.4667
Epoch 7/150
90/90 [==============================] - 0s 66us/step - loss: 1.0301 - acc: 0.6259 - val_loss: 1.1556 - val_acc: 0.5667
Epoch 8/150
90/90 [============

In [10]:
model.predict(X_test)[:5], y_test[:5]

(array([[0.03754101, 0.7026595 , 0.25979954],
        [0.984421  , 0.01389271, 0.00168624],
        [0.00346922, 0.1273421 , 0.8691887 ],
        [0.03193707, 0.5803498 , 0.38771313],
        [0.02681728, 0.632687  , 0.34049574]], dtype=float32),
 array([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=float32))

In [11]:
## accuracy of keras model
np.sum(np.array(model.predict(X_train) > 0.5).astype(float) == y_train) / y_train.shape[0] / y_train.shape[1]

0.9666666666666667

In [12]:
np.sum(np.array(model.predict(X_test) > 0.5).astype(float) == y_test) / y_test.shape[0] / y_test.shape[1]

0.9733333333333333

In [13]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=base_model, epochs=150, batch_size=32, verbose=1)
estimator.fit(X_train, y_train)
# results = cross_val_score(model, X, dummy_y, cv=kfold, scoring=accuracy_score)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/150
100/100 [==============================] - 0s 3ms/step - loss: 2.0960 - acc: 0.5600
Epoch 2/150
100/100 [==============================] - 0s 99us/step - loss: 1.9287 - acc: 0.5600
Epoch 3/150
100/100 [==============================] - 0s 104us/step - loss: 1.7702 - acc: 0.5600
Epoch 4/150
100/100 [==============================] - 0s 139us/step - loss: 1.6132 - acc: 0.5600
Epoch 5/150
100/100 [==============================] - 0s 104us/step - loss: 1.4676 - acc: 0.5600
Epoch 6/150
100/100 [==============================] - 0s 159us/step - loss: 1.3314 - acc: 0.5600
Epoch 7/150
100/100 [==============================] - 0s 114us/step - loss: 1.2013 - acc: 0.5600
Epoch 8/150
100/100 [==============================] - 0s 109us/step - loss: 1.0934 - acc: 0.5600
Epoch 9/150
100/100 [==============================] - 0s 74us/step - loss: 0.9931 - acc: 0.5600
Epoch 10/150
100/100 [==============================] - 0s 119us/step - loss: 0.9155 - acc: 0.5600
Epoch 11/150
100/100 [=

In [15]:
accuracy_score(y, estimator.predict(X))

150/150 [==============================] - 0s 301us/step


0.82

In [3]:
# print(f'accuracy score {accuracy_score(y_train, estimator.predict(X_train)).round(2)}')
# print(f'accuracy score {accuracy_score(y_test, estimator.predict(X_test)).round(2)}')

In [24]:
print(f'train loss+score: {model.evaluate(X_train, y_train)},\n test loss+score: {model.evaluate(X_test, y_test)}')

50/50 [==============================] - 0s 69us/step
train loss+score: [0.19410236120224, 0.9666666412353515],
 test loss+score: [0.16639681816101073, 0.9733333539962769]
